In [ ]:
#Creates initial condition file with temperature and salinities
%reset
import numpy as np
import scipy.io as sio
import netCDF4 as nc
import matplotlib.pyplot as plt
%pwd

In [ ]:
#SPECIFY THESE:
XN = 640 #NUMBER OF POINTS IN X
YN = 4  #NUMBER OF POINTS IN Y
ZN = 120 #NUMBER OF POINTS IN Z #default 120
perturbation = np.expand_dims(np.random.random([ZN,YN,XN])*.000001,axis=0)

depthmat=nc.Dataset('depthmat.nc')['depthmat'][:]
xmat = nc.Dataset('xmat.nc')['xmat'][:]
zmat = nc.Dataset('zmat.nc')['zmat'][:]
smat = nc.Dataset('smat.nc')['smat'][:]
tmat = nc.Dataset('tmat.nc')['tmat'][:]

interpx1=xmat
interpz1=zmat
interpx2=np.linspace(0,75000,XN)
interpz2=np.linspace(0,2500,ZN)

sinterp=np.expand_dims(np.interp(interpz2,interpz1,smat),axis=0)
sinterp=np.expand_dims(sinterp,axis=2)
sinterp=np.expand_dims(sinterp,axis=3)
s_input=np.repeat(sinterp,YN,axis=2)
s_input=np.repeat(s_input,XN,axis=3)+perturbation

tinterp=np.expand_dims(np.interp(interpz2,interpz1,tmat),axis=0)
tinterp=np.expand_dims(tinterp,axis=2)
tinterp=np.expand_dims(tinterp,axis=3)
t_input=np.repeat(tinterp,YN,axis=2)
t_input=np.repeat(t_input,XN,axis=3)+perturbation

YC = np.linspace(0.0625,0.4375,YN) #0.5km in y
XC = np.linspace(0.0+80.0/(XN*2),80.0-80.0/(XN*2),XN) #80km in x
ZC=np.linspace(0,2499.8179,ZN)


In [ ]:
print(s_input.shape)
print(perturbation.shape)

In [ ]:
X, Z = np.meshgrid(interpx2/1000, interpz2)
fig=plt.figure(figsize=(7, 4), dpi= 80, facecolor='w', edgecolor='k')
img1=plt.pcolor(X, Z, s_input[0,:,0,:])
plt.gca().invert_yaxis()
plt.title('Salinity', fontsize=14)
plt.xlabel('X Position (km)',fontsize=14)
plt.ylabel('Depth (m)',fontsize=14)
plt.colorbar(img1)
plt.show()

fig=plt.figure(figsize=(7, 4), dpi= 80, facecolor='w', edgecolor='k')
img2=plt.pcolor(X, Z, t_input[0,:,0,:],vmin=-.89,vmax=0.006)
plt.gca().invert_yaxis()

plt.title('Temperature', fontsize=14)
plt.xlabel('X Position (km)',fontsize=14)
plt.ylabel('Depth (m)',fontsize=14)
plt.colorbar(img2)
plt.show()
print(np.max(s_input))
print(np.min(s_input)) 
print(np.max(t_input))
print(np.min(t_input)) 

In [ ]:
writing = nc.Dataset("IC_TS.nc","w",format="NETCDF3_64BIT_OFFSET")
TIME = writing.createDimension("TIME",1)
DEPTH = writing.createDimension("DEPTH",ZN)
LAT = writing.createDimension("LAT",YN)
LON = writing.createDimension("LON",XN)


LAT = writing.createVariable("LAT","f8",("LAT"))
LAT[:]=YC
LAT.standard_name = "YC (y coordinate at center of grid cell)"
LAT.units = "km"
LAT.cartesian_axis = "Y"

    
LON = writing.createVariable("LON","f8",("LON"))
LON[:]=XC
LON.standard_name = "XC (x coordinate at center of grid cell)"
LON.units = "km"
LON.cartesian_axis = "X" ;

TIME = writing.createVariable("TIME","f8",("TIME"))
TIME[:]=0.0
TIME.standard_name = "time"
TIME.units = "days since 0001-01-01 00:00:00"
TIME.calendar = "noleap" ;
TIME.cartesian_axis = "T" ;
    
DEPTH = writing.createVariable("DEPTH","f8",("DEPTH")) #FOR Z COORDINATES
DEPTH[:]=ZC
DEPTH.standard_name = "Z coordinate"
DEPTH.units = "m"
DEPTH.cartesian_axis = "Z" ;

PTEMP = writing.createVariable("PTEMP","f4",("TIME","DEPTH","LAT","LON"),fill_value=-1.e+34)
PTEMP[:,:,:,:]=t_input
PTEMP.standard_name = "Initial potential temperature referenced to 0dbar"
PTEMP.units = "Celsius"

SALT = writing.createVariable("SALT","f4",("TIME","DEPTH","LAT","LON"),fill_value=-1.e+34)
SALT[:,:,:,:]=s_input
SALT.standard_name = "Initial salinity"
SALT.units = "PSU"


writing.close()

In [ ]:
#NEW FILE CHECK
filenew = nc.Dataset('IC_TS.nc')
print(filenew)
temp_new=filenew.variables['PTEMP'][:]
print(temp_new.shape)
salt_new=filenew.variables['SALT'][:]
print(salt_new.shape)